### Getting the files & import statements ###

In [1]:
import pandas as pd
import numpy as np
import os

In [22]:
# Get participant number 
participant_number = input("Enter participant number: ")

base_path = "/Users/alina/Downloads/Thesis/raw/"

# Folder path for the participant
folder_path = os.path.join(base_path, f"Participant {participant_number}")

# Get recording number 
recording_number = input("Enter recording number: ")

# Recording folder path
recording_path = os.path.join(folder_path, f"P{participant_number}_{recording_number}")

# Get files
world_timestamps_file = os.path.join(recording_path, "world_timestamps.csv")
detections_file = os.path.join(recording_path, f"detections_{participant_number}_{recording_number}.csv")
fixations_file = os.path.join(recording_path, "fixations.csv")
face_file = os.path.join(recording_path, "fixations_on_face.csv")

# Load as dfs
world_timestamps_df = pd.read_csv(world_timestamps_file) #contains timestamps for each video frame
detections_df = pd.read_csv(detections_file) #contains detections from YOLO
fixations_df = pd.read_csv(fixations_file) #fixations data
face_df = pd.read_csv(face_file) #fixations on face data from FaceMapper

### Matching video frames and timestamps

In [23]:
## function to see if the gaze is inside the bounding box of an object
def is_gaze_in_bbox(gaze_x, gaze_y, x_min, y_min, x_max, y_max):
    return x_min <= gaze_x <= x_max and y_min <= gaze_y <= y_max

## function to prioritise the smallest object fixated
def filter_objects_by_area(group):
    group = group.copy()
    group["area"] = (group["X_max"] - group["X_min"]) * (group["Y_max"] - group["Y_min"])

    # Find the minimum area
    min_area = group["area"].min()
    
    # Get all objects with the minimum area
    smallest_objects = group[group["area"] == min_area]
    
    if len(smallest_objects) == 1:
        return smallest_objects
    else:
        person_objects = smallest_objects[smallest_objects["Class"] == "person"]
        if not person_objects.empty:
            # Return only one person object (the first one)
            return person_objects.iloc[[0]]

    # If no person among smallest, return just one of the smallest objects
    return smallest_objects.iloc[[0]]

In [24]:
## world_timestamps.csv file from pupil cloud represents timestamps for each video frame
# merge_asof is like a nearest match join for ordered time-series data

# renaming so all timestamp columns are named the same to merge the dfs
fixations_df["timestamp [ns]"] = fixations_df["start timestamp [ns]"]

## making a video_df which contains matching frames and ts
ts = world_timestamps_df['timestamp [ns]']

video_df = pd.DataFrame({
        "Frame": np.arange(len(world_timestamps_df)),
        "timestamp [ns]": ts,
    })

# Make sure everything is sorted by timestamp so merge_asof works properly
fixations_df = fixations_df.sort_values("start timestamp [ns]")
world_timestamps_df = world_timestamps_df.sort_values("timestamp [ns]")

# Merge-asof to assign the closest frame_id to each fixation
fixations_with_frame = pd.merge_asof(
    fixations_df,
    video_df,
    on="timestamp [ns]",
    direction="nearest",  # to match the closest timestamp from video_df, whether it's before or after the fixation timestamp
) 

# merging to assign fixations to frames
fixations_with_detections = pd.merge(
    fixations_with_frame,
    detections_df,
    on="Frame",
    how="left" # keep all the fixations, even if a frame has no detection => nan
)

# calculate in_bbox 
fixations_with_detections["in_bbox"] = fixations_with_detections.apply(
    lambda row: is_gaze_in_bbox(
        row["fixation x [px]"], row["fixation y [px]"],
        row["X_min"], row["Y_min"], row["X_max"], row["Y_max"]
    ),
    axis=1
)

# filter only fixations that match at least one bounding box
in_bbox_fixations = fixations_with_detections[fixations_with_detections["in_bbox"] == True]

#  group by fixation ID and drop "person" if there is another class in same fixation group
filtered_fixations = in_bbox_fixations.groupby(
    "Frame", group_keys=False # grouping on frame as i need fixation id column later
).apply(filter_objects_by_area, include_groups=False)

fixations_on_persons = filtered_fixations.loc[filtered_fixations["Class"] == "person"]
fixations_on_persons.head()

,section id,recording id,fixation id,start timestamp [ns],end timestamp [ns],duration [ms],fixation x [px],fixation y [px],azimuth [deg],elevation [deg],timestamp [ns],X_min,Y_min,X_max,Y_max,Confidence,Class,in_bbox,area
248,b1c71554-ecfb-44b0-b602-460c5abaa08e,d0b291d0-0bd5-442a-8423-fec772d30673,21,1749735372418762303,1749735372528886303,110,1079.044,738.357,16.642094,-8.314045,1749735372418762303,952.549927,369.262543,1599.192139,1199.077148,0.251754,person,True,536593.152117
320,b1c71554-ecfb-44b0-b602-460c5abaa08e,d0b291d0-0bd5-442a-8423-fec772d30673,26,1749735375036257303,1749735375156378303,120,482.510,492.018,-21.804647,7.181194,1749735375036257303,393.745880,402.144318,520.987427,664.209717,0.297828,person,True,33345.606709
331,b1c71554-ecfb-44b0-b602-460c5abaa08e,d0b291d0-0bd5-442a-8423-fec772d30673,27,1749735375176377303,1749735375371502303,195,487.846,492.052,-21.462604,7.185133,1749735375176377303,430.235321,399.290222,492.676666,504.847046,0.355564,person,True,6591.110070
345,b1c71554-ecfb-44b0-b602-460c5abaa08e,d0b291d0-0bd5-442a-8423-fec772d30673,28,1749735375396502303,1749735375486626303,90,490.994,512.267,-21.226457,5.924220,1749735375396502303,438.778717,401.282715,535.685364,656.633545,0.408904,person,True,24745.192682
456,b1c71554-ecfb-44b0-b602-460c5abaa08e,d0b291d0-0bd5-442a-8423-fec772d30673,36,1749735381462231303,1749735381637370303,175,997.378,430.683,11.436013,11.232261,1749735381462231303,949.451538,422.517334,997.453552,618.104797,0.714155,person,True,9388.592187


### Creating a new file with categorised fixations

In [25]:
# filtering to only fixations on bodies
fixations_on_persons = fixations_on_persons[fixations_on_persons["in_bbox"] == True]

# initialising a df for all types of fixations
categorized_fixations_all = []

# using FaceMapper file to get all the basic fixations information
for _, row in face_df.iterrows():
    timestamp = row['start timestamp [ns]']
    fixation_id = row['fixation id']
    fix_duration = (row['end timestamp [ns]'] - row['start timestamp [ns]'])/1000000     #converting to ms
    fixation_type = "background" #make default type

    # face fixations
    if row.get('fixation on face') == True:
        fixation_type = "face"

    # body fixations (only if not already labeled as face)
    elif not fixations_on_persons[fixations_on_persons['timestamp [ns]'] == timestamp].empty:
        fixation_type = "body"
        matching_row = fixations_on_persons[fixations_on_persons['timestamp [ns]'] == timestamp].iloc[0]
        fixation_id = matching_row['fixation id']
        fix_duration = matching_row['duration [ms]']

    categorized_fixations_all.append({
        'timestamp [ns]': timestamp,
        'type': fixation_type,
        'fixation id': fixation_id,
        'duration [ms]': fix_duration
    })

# saving a file with fixations categorised by face, body and background
all_categorized_fixations_df = pd.DataFrame(categorized_fixations_all)
saving_path = os.path.join(recording_path, f"fixations_on_everything_{participant_number}_{recording_number}.csv")
all_categorized_fixations_df.to_csv(saving_path, index=False)

all_categorized_fixations_df.head()

,timestamp [ns],type,fixation id,duration [ms]
0,1749735362539546303,background,1,95.125
1,1749735363115169303,background,2,80.000
2,1749735363215294303,background,3,75.000
3,1749735363950916303,background,4,195.125
4,1749735364936788303,background,5,95.124
